In [ ]:
#pip install apify-client

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
pd.set_option('display.max_columns', None)

In [2]:
tiktok = pd.read_csv('tiktok_dataset.csv')
#I got this data from kaggle: "https://www.kaggle.com/datasets/yakhyojon/tiktok"

In [3]:
tiktok.head()

,#,claim_status,video_id,video_duration_sec,video_transcription_text,verified_status,author_ban_status,video_view_count,video_like_count,video_share_count,video_download_count,video_comment_count
0,1,claim,7017666017,59,someone shared with me that drone deliveries a...,not verified,under review,343296.0,19425.0,241.0,1.0,0.0
1,2,claim,4014381136,32,someone shared with me that there are more mic...,not verified,active,140877.0,77355.0,19034.0,1161.0,684.0
2,3,claim,9859838091,31,someone shared with me that american industria...,not verified,active,902185.0,97690.0,2858.0,833.0,329.0
3,4,claim,1866847991,25,someone shared with me that the metro of st. p...,not verified,active,437506.0,239954.0,34812.0,1234.0,584.0
4,5,claim,7105231098,19,someone shared with me that the number of busi...,not verified,active,56167.0,34987.0,4110.0,547.0,152.0


In [4]:
data.shape

(19382, 12)

In [5]:
#here I had to use two APIs, the first one scrapes random usernames on Instagram, 
#and the second one scrapes reels info from these accounts
#so for the first API, it needed a keyword, so I included a letter of Alphabet, for each keyword it actually gives about 50 outputs.
#I get a lot of unwanted columns here about the users, I'll be only using the username column

import string
from apify_client import ApifyClient

# Initialize ApifyClient
apify_client = ApifyClient('enter your api key')

# Initialize an empty DataFrame to store all users
all_users = pd.DataFrame()

# Combine lowercase and uppercase letters
letters = string.ascii_lowercase + string.ascii_uppercase

# Loop over each letter in the alphabet
for letter in letters:
    # Call the API with the current letter as the keyword
    actor_run = apify_client.actor('apidojo/instagram-user-scraper').call(
        run_input={
            "customMapFunction": "(object) => { return {...object} }",
            "getFollowers": False,
            "getFollowings": False,
            "includeUnavailableUsers": False,
            "keyword": letter,
            "maxItems": 5000,
            "startUrls": [
                "https://www.instagram.com/jessandquinn/"
            ]
        }
    )
    # Get the dataset items
    dataset_items = apify_client.dataset(actor_run['defaultDatasetId']).list_items().items
    
    # Convert the results to a DataFrame
    users = pd.DataFrame(dataset_items)
    
    # Append the results to the all_users DataFrame
    all_users = pd.concat([all_users, users], ignore_index=True)

In [6]:
# Print the username column
if 'username' in all_users.columns:
    for user in all_users['username']:
        print(user)
else:
    print("No usernames found.")

jessandquinn
aliaabhatt
andreeabostanica
turkialalshik
aitym
roses_are_rosie
arohikhurana_
arianagrande
anyataylorjoy
ashokleyland_official
official.anuragthakur
lakers
alarabytv2
aebyborntoshine
haleyybaylee
_amalashaji_
taranadarsh
haniaheheofficial
antonelaroccuzzo
ayoedebiri
imamhussainorg
ardaguler
theanthonyedwards_
lunin_andrey
staskaranikolaou
xtina
ananddeverakonda
amazonminitv
fafdup
ankit_baiyanpuria
ajplusarabi
alhilal
alexiaputellas
anuragxcricket
aitanabonmati
anastasile
abdulkerimbardakci
aditiraohydari
ameliadimz
alnassr
goar_avetisyan
abhisheksharma_4
angelreese5
anthonyjoshua
la_sparks
anna_paull
applemusic
aselmustafaeva
ashanti
ayeshaakhan_official
jessandquinn
bobbi
britneyspears
bharti.laughterqueen
cznburak
erling.haaland
burcuozberk
bamgyuuuu
byeonwooseok
justinbieber
baderalsafar
haileybieber
bbciplayer
atalantabc
botafogo
bvb09
selcuk79
graciebon
brittanylynne
gnabnahc
baku.ws_official
brahim
seyi_vibez
bleacherreport
brunabiancardi
quenblackwell
judebellingha

In [8]:
df_unique = all_users.drop_duplicates(subset='username')

In [12]:
reel_run = apify_client.actor('apify/instagram-reel-scraper').call(
    run_input={
        "resultsLimit": 5,
        "username": df_unique['username'].tolist()
    }
)

In [14]:
dataset_items1 = apify_client.dataset(reel_run['defaultDatasetId']).list_items().items
    
# Convert the results to a DataFrame
reels = pd.DataFrame(dataset_items1)

In [15]:
reels.head()

,inputUrl,id,type,shortCode,caption,hashtags,mentions,url,commentsCount,firstComment,latestComments,dimensionsHeight,dimensionsWidth,displayUrl,images,videoUrl,alt,likesCount,videoViewCount,videoPlayCount,timestamp,childPosts,ownerFullName,ownerUsername,ownerId,productType,videoDuration,isSponsored,taggedUsers,musicInfo,coauthorProducers,locationName,locationId,isPinned,paidPartnership,sponsors,error,description
0,https://www.instagram.com/turkialalshik,3377458185513934133,Video,C7fJFM_Sx01,Deontay Wilder’s right hand is out of this wor...,"[QueensberryVsMatchroom, 5vs5, RiyadhSeason]",[turkialalshik],https://www.instagram.com/p/C7fJFM_Sx01/,96.0,نعم كلام صحيح، فإذا ضرب ديونتي ويلدر اي شخص بي...,"[{'id': '18023032144993896', 'text': 'نعم كلام...",1333.0,750.0,https://scontent.cdninstagram.com/v/t51.29350-...,[],https://scontent.cdninstagram.com/v/t50.2886-1...,NaN,10439.0,20250.0,98264.0,2024-05-27T21:14:07.000Z,[],DAZN Boxing,daznboxing,3408671540,clips,20.431,False,[{'full_name': 'TURKI ALALSHIKH تركي آل الشيخ...,"{'artist_name': 'daznboxing', 'song_name': 'Or...","[{'id': '7020673524', 'is_verified': True, 'pr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.instagram.com/jessandquinn,3377480015372241910,Video,C7fOC3oPkv2,…,"[comedy, funny, funnyvideos, viral, explorepag...",[],https://www.instagram.com/p/C7fOC3oPkv2/,10.0,You look like you are dating Zendaya,"[{'id': '17983409876677583', 'text': 'You look...",1333.0,750.0,https://scontent.cdninstagram.com/v/t51.29350-...,[],https://scontent.cdninstagram.com/v/t66.30100-...,NaN,677.0,3406.0,13553.0,2024-05-27T22:02:01.000Z,[],Jess and Quinn,jessandquinn,56559259167,clips,6.066,False,NaN,"{'artist_name': 'jessandquinn', 'song_name': '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.instagram.com/jessandquinn,3375973316207480075,Video,C7Z3dgrvo0L,I am speed…,[],[],https://www.instagram.com/p/C7Z3dgrvo0L/,22.0,I can't do that but If I could...,"[{'id': '17878091439080353', 'text': 'I can't ...",1920.0,1080.0,https://scontent.cdninstagram.com/v/t51.29350-...,[],https://scontent.cdninstagram.com/v/t66.30100-...,NaN,2011.0,16248.0,40425.0,2024-05-25T20:07:09.000Z,[],Jess and Quinn,jessandquinn,56559259167,clips,10.300,False,NaN,"{'artist_name': 'jessandquinn', 'song_name': '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.instagram.com/aitym,3375585127052257327,Video,C7YfMnXCqgv,"“Когда руки в кp0Bи, запачкаться деpьMoм не пр...","[Бишимбаев, засалтанат, салтанат]",[aitym],https://www.instagram.com/p/C7YfMnXCqgv/,93.0,👏,"[{'id': '17991000041643958', 'text': '👏', 'own...",1333.0,750.0,https://scontent.cdninstagram.com/v/t51.29350-...,[],https://scontent.cdninstagram.com/v/t66.30100-...,NaN,5256.0,128065.0,450969.0,2024-05-25T07:15:21.000Z,[],Timur Balymbetov,balukz,29814185,clips,55.866,False,"[{'full_name': 'Aitbek Amangeldi', 'id': '1975...","{'artist_name': 'balukz', 'song_name': 'Origin...","[{'id': '197584566', 'is_verified': True, 'pro...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.instagram.com/aitym,3367975217018611095,Video,C69c52WtqWX,15.05.1992 - ♾️❤️\nС днем рождения Салтик. \n\...,[],[],https://www.instagram.com/p/C69c52WtqWX/,13897.0,❤️,"[{'id': '17901891818913006', 'text': '❤️', 'ow...",1920.0,1080.0,https://scontent.cdninstagram.com/v/t51.29350-...,[],https://scontent.cdninstagram.com/v/t50.2886-1...,NaN,330885.0,2215143.0,5733459.0,2024-05-14T19:13:40.000Z,[],Фонд памяти Салтанат Нукеновой,saltanat.foundation,66305780642,clips,85.400,False,"[{'full_name': 'Aitbek Amangeldi', 'id': '1975...","{'artist_name': 'saltanat.foundation', 'song_n...","[{'id': '197584566', 'is_verified': True, 'pro...","Astana, Kazakhstan",213709399,NaN,NaN,NaN,NaN,NaN


In [16]:
reels.to_csv('insta_reels.csv', index=False)